# Data exploratory analysis

The goal of this notebook is to explore the data and identify potential interesting insights

In [57]:
import pandas as pd
import requests as req
import numpy as np

# Scrapping packages
from bs4 import BeautifulSoup as bs

# Google Trends API connection
# documentation -> https://pypi.org/project/pytrends/#api-methods
from pytrends.request import TrendReq
pytrends = TrendReq()

# Statistics + plotting
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm


##### CLEAN IT

## Calculate min, avg etx

In [ ]:
# Populate the DataFrames with seach volumes
for sub in sub_dic:
    
    # Create new columns
    sub_dic[sub] = (sub_dic[sub]
           .assign(Min=round((search_vol[sub]['min']*12) / 52))
           .assign(Max=round((search_vol[sub]['max']*12) / 52))
           .assign(Average=round((((search_vol[sub]['min'] * 12) / 52) + ((search_vol[sub]['max']*12)/52)) / 2))
               )
    
    # Generate new column with random per week
    sub_dic[sub]['Random'] = [np.random.randint(sub_dic[sub]['Min'].min(), sub_dic[sub]['Max'].max()) for row in range(len(sub_dic[sub]))]
    
    # Create a colum trend * mean
    sub_dic[sub]['Search Vol'] = round((sub_dic[sub]['Trend'] / 100) * sub_dic[sub]['Average'], 0)

In [ ]:
# Check the output
sub_dic['a_4'].head()

# Basic plotting

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Volume of searchs
sub_colors = {'presentation': 'red', 
              'a_4': 'orange', 
              'letter': 'blue', 
              'letterhead': 'grey',
              'resume': 'black',
              'presentation_4_3': 'yellow',
              'report': 'purple',
              'daily_report': 'brown',
              'memo': 'green',
              'invoice': 'magenta'}

plt.figure(figsize=(15,5))

for sub, colr in zip(sub_dic, sub_colors.values()):
    
    # Create a graph with the volume of searches
    ax = sns.lineplot(data=sub_dic[sub], 
                  x='Date', 
                  y='Search Vol',
                  color=colr)
    
# [Bonus] --> Adding legen with sub category name

Resume and Invoice sub-categories have much more search vol than the rest. For the sake of the analysis, could be interesting to add those sub-categories with less than 10.000 monthly visits.

In [ ]:
# Same plot with subcategories with less than 10.000 monthly searches on average
sub_dic_gp = {'Under 10k': pd.DataFrame(columns=['Date', 'Search Vol']),
              'Under 100k': pd.DataFrame(columns=['Date', 'Search Vol'])}

# Aggregate DataFrames by groups depending on Search Vol.
for sub, colr in zip(sub_dic, sub_colors.values()):
    
    # Subcategories with an avge of less than 10k monthly seaches
    if (sub_dic[sub]['Search Vol'].mean() * 52) / 12 < 10000:
        sub_dic_gp['Under 10k'] = pd.concat([sub_dic_gp['Under 10k'], sub_dic[sub]])
    
    # Subcategories with an avge of less than 10k monthly seaches
    elif ((sub_dic[sub]['Search Vol'].mean() * 52) / 12 < 100000) & ((sub_dic[sub]['Search Vol'].mean() * 52) / 12 > 10000):
        sub_dic_gp['Under 100k'] = pd.concat([sub_dic_gp['Under 100k'], sub_dic[sub]])
    
    # The rest will be analysed individually
    else:
        sub_dic_gp[sub] = sub_dic[sub]

# Grouping by day and summing 'Seatch Vol.'
sub_dic_gp['Under 10k'] = sub_dic_gp['Under 10k'].groupby('Date', as_index=False).agg('sum')
sub_dic_gp['Under 100k'] = sub_dic_gp['Under 100k'].groupby('Date', as_index=False).agg('sum')

In [ ]:
# Checking the results
sub_dic_gp['Under 100k'].head()

Now we have a dictionary with a DataFrame that aggregates the subcategories with less that 10k monthly searches as 'Under 10k', a DataFrame that aggregates the subcategories with less that 100k monthly searches as 'Under 100k' and individual DataFrames for those subcategories with more than 100k monthly searches.

In [ ]:
# Plot 
plt.figure(figsize=(15,5))

for sub, colr in zip(sub_dic_gp, sub_colors.values()):
    
    # Create a graph with the volume of searches
    ax = sns.lineplot(data=sub_dic_gp[sub], 
                  x='Date', 
                  y='Search Vol',
                  color=colr)
    
## [BONUS] - Add legends

It would be useful to list the subcategories in each category.

In [ ]:
# [BONUS] - List a summary with the categories in each bin. [Under 10k] | [Under 100k] | [Over 100k]

### Find 'Search Vol' trend

In [ ]:
# Combine all searches and find the trend. 
# We will be able to compare each sub_category with the overall trend
# Statistical analysis

In [ ]:
sub_dic_gp['Total'] = pd.DataFrame(columns=['Date', 'Search Vol'])

# Concat all DataFrames into one
for sub in sub_dic:
    sub_dic_gp['Total'] = pd.concat([sub_dic_gp['Total'], sub_dic[sub]])

sub_dic_gp['Total'] = sub_dic_gp['Total'].groupby('Date', as_index=False).agg('sum')

#### Calculate the fitted line

In [ ]:
# Create a constant
sub_dic_gp['Total'] = sm.add_constant(sub_dic_gp['Total'])
sub_dic_gp['Total'] = sub_dic_gp['Total'].assign(t=range(1, len(sub_dic_gp['Total']) + 1))
# sub_dic_gp['Total'] = sub_dic_gp['Total'].assign(t2=range(1, (len(sub_dic_gp['Total']) + 1)**2))

# Run OLS
X = sub_dic_gp['Total'][["const", "t"]]
y = sub_dic_gp['Total']["Search Vol"]

lin_reg = sm.OLS(y, X)
results_total = lin_reg.fit()

results_total.summary()

In [ ]:
sub_dic_gp['Total'] = sub_dic_gp['Total'].assign(y_hat=results_total.predict(X))

plt.figure(figsize=(15,5))
    
sns.scatterplot(data=sub_dic_gp['Total'], 
                x='t', 
                y='Search Vol');

sns.lineplot(data=sub_dic_gp['Total'],
             x='t',
             y="y_hat",
             color="green");

In [ ]:
# Run OLS
lin_reg = sm.OLS(sub_dic_gp['Total']["Date"],
                 sub_dic_gp['Total'][['const', 'Search Vol']])

result = lin_reg.fit()

corr = sm.add_constant(corr)
corr.head()

# Run OLS
lin_reg = sm.OLS(corr["Search Vol (avge)"],
                 corr[["const", "Number of templates"]])

result = lin_reg.fit()

result.summary()
# result.summary()

In [ ]:
plt.figure(figsize=(15,5))
    
    # Create a graph with the total volume of searches
ax = sns.lineplot(data=sub_dic_gp['Total'], 
                  x='Date', 
                  y='Search Vol',)

ax = sns.regplot(data=sub_dic_gp['Total'], 
                  x='Date', 
                  y='Search Vol',)

In [ ]:
# Find if there is a correlation between 'Templates' that Canva is offering and 'Search Vol' 
# We will be able to do a linear regression
# Statistical analysis

### 'Search Vol' vs. # 'Template '

Is there a correlation between the Search Vol for the Keywords assigned to the category and the number of templates that Canva is offering? Is it statistically significant?

In [ ]:
corr = pd.DataFrame(columns=['Sub-Category', 'Search Vol (avge)', 'Number of templates'])

corr['Sub-Category'] = sub_dic.keys()
corr['Search Vol (avge)'] = [int(sub_dic[sub]['Search Vol'].mean()) for sub in sub_dic]
corr['Number of templates'] = [int(sub_dic[sub]['Number of templates'].mean()) for sub in sub_dic]

corr

In [ ]:
# Scatter plot to infere if it makes sense to calculate the fitted line
plt.figure(figsize=(15,5))
    
sns.scatterplot(data=corr, 
                x='Number of templates', 
                y='Search Vol (avge)',
                hue='Sub-Category');

In [ ]:
# See if there is a correlation between the two variables
corr = sm.add_constant(corr)
corr.head()

# Run OLS
lin_reg = sm.OLS(corr["Search Vol (avge)"],
                 corr[["const", "Number of templates"]])

result = lin_reg.fit()

result.summary()

In [ ]:
corr = corr.assign(y_hat=result.predict(corr[["const", "Number of templates"]]))

In [ ]:
plt.figure(figsize=(15,5))
    
sns.scatterplot(data=corr, 
                x='Number of templates', 
                y='Search Vol (avge)',
                hue='Sub-Category');

sns.lineplot(data=corr,
             x='Number of templates',
             y="y_hat",
             color="g");